In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as py

# Merging World GDP and World Population
1. Ensure 1:1 mapping between NOC:Team to facilitate merging with World GDP and Population data
2. Use noc_regions.csv as a key to map team:country
3. Manually fill out the three countries with no Country Name
4. Import and Clean GDP Data
5. Merge GDP Data
6. Import and Clean World Population Data
7. Merge World Population Data and manually fill missing values

In [2]:
male = pd.read_csv('clean_male_df.csv')
female = pd.read_csv('clean_female_df.csv')
male['Team'].value_counts()

United States    763
Australia        652
Great Britain    527
Germany          496
Russia           486
                ... 
Japan-1            2
Germany-1          2
Thailand-2         2
Italy-1            1
Italy-2            1
Name: Team, Length: 254, dtype: int64

# Make every team map to one country

> There are these few countires in both Male and Female DF whose NOC deosnt match to a unique Team

In [13]:
print(male.loc[:, ['NOC', 'Team']].drop_duplicates()['NOC'].value_counts())

CHN    3
SRB    3
AUS    3
INA    3
MAS    3
      ..
FSM    1
ANG    1
MEX    1
BUR    1
MTN    1
Name: NOC, Length: 210, dtype: int64


In [26]:
print(female.loc[:, ['NOC', 'Team']].drop_duplicates()['NOC'].value_counts())

CHN    4
DEN    3
JPN    3
CAN    3
SVK    3
      ..
SSD    1
SUD    1
ANG    1
MEX    1
MAW    1
Name: NOC, Length: 210, dtype: int64


> Import NOC_Country key to map each Team to a specific country, since NOC:Country is not a 1:1 Mapping

In [27]:
noc_country = pd.read_csv('noc_regions.csv')
noc_country.drop('notes', axis = 1 , inplace = True)
noc_country.rename(columns = {'region':'Country'}, inplace = True)
noc_country

,NOC,Country
0,AFG,Afghanistan
1,AHO,Curacao
2,ALB,Albania
3,ALG,Algeria
4,AND,Andorra
...,...,...
225,YEM,Yemen
226,YMD,Yemen
227,YUG,Serbia
228,ZAM,Zambia


> Add a country column to the Olympic Data (male,female)

In [28]:
male_merge = male.merge(noc_country, left_on = 'NOC', right_on = 'NOC', how = 'left')
female_merge = female.merge(noc_country, left_on = 'NOC', right_on = 'NOC', how = 'left')

In [29]:
display(male_merge.loc[male_merge['Country'].isnull(),['NOC', 'Team']].drop_duplicates())
display(female_merge.loc[female_merge['Country'].isnull(),['NOC', 'Team']].drop_duplicates())

,NOC,Team
153,ROT,Refugee Olympic Athletes
601,SGP,Singapore
11523,TUV,Tuvalu


,NOC,Team
368,SGP,Singapore
1997,ROT,Refugee Olympic Athletes
9272,TUV,Tuvalu


In [30]:
print(male.Team.value_counts())
print('-'*100)
print(male_merge.Country.value_counts())
print('-'*100)
print(female.Team.value_counts())
print('-'*100)
print(female_merge.Country.value_counts())

United States    763
Australia        652
Great Britain    527
Germany          496
Russia           486
                ... 
Japan-1            2
Germany-1          2
Thailand-2         2
Italy-1            1
Italy-2            1
Name: Team, Length: 254, dtype: int64
----------------------------------------------------------------------------------------------------
USA              783
Australia        658
UK               547
Germany          500
Russia           494
                ... 
Chad               3
Liechtenstein      3
Bhutan             3
South Sudan        2
Kosovo             2
Name: Country, Length: 205, dtype: int64
----------------------------------------------------------------------------------------------------
United States           825
China                   620
Australia               610
Germany                 536
Russia                  518
                       ... 
Belarus-2                 2
Hungary-1                 2
Monaco                    1
Nethe

# Still remains these 3 countries with no Country Name, manually fill them out

In [31]:
male_merge.loc[male_merge['Country'].isnull(),['NOC', 'Team']].drop_duplicates()

,NOC,Team
153,ROT,Refugee Olympic Athletes
601,SGP,Singapore
11523,TUV,Tuvalu


In [32]:
female_merge.loc[female_merge['Country'].isnull(),['NOC', 'Team']].drop_duplicates()

,NOC,Team
368,SGP,Singapore
1997,ROT,Refugee Olympic Athletes
9272,TUV,Tuvalu


In [33]:
print(f"Rows with missing countries in male data: {male_merge['Country'].isna().sum()}")
print(f"Rows with missing countries in female data: {female_merge['Country'].isna().sum()}")

Rows with missing countries in male data: 40
Rows with missing countries in female data: 49


In [34]:
male_merge['Country'] = np.where(male_merge['NOC']=='SGP', 'Singapore', male_merge['Country'])
male_merge['Country'] = np.where(male_merge['NOC']=='ROT', 'Refugee Olympic Athletes', male_merge['Country'])
male_merge['Country'] = np.where(male_merge['NOC']=='TUV', 'Tuvalu', male_merge['Country'])

female_merge['Country'] = np.where(female_merge['NOC']=='SGP', 'Singapore', female_merge['Country'])
female_merge['Country'] = np.where(female_merge['NOC']=='ROT', 'Refugee Olympic Athletes', female_merge['Country'])
female_merge['Country'] = np.where(female_merge['NOC']=='TUV', 'Tuvalu', female_merge['Country'])


> Rename Country column to Team

In [35]:

male_merge.drop('Team', axis = 1, inplace = True)
male_merge.rename(columns = {'Country': 'Team'}, inplace = True)

female_merge.drop('Team', axis = 1, inplace = True)
female_merge.rename(columns = {'Country': 'Team'}, inplace = True)

In [36]:
male_merge.loc[male_merge['Team'].isnull(),['NOC', 'Team']].drop_duplicates()

,NOC,Team


In [37]:
male_merge.head()

,Unnamed: 0,ID,Name,Sex,Age,Height,Weight,NOC,Games,Year,Season,City,Sport,Event,Medal,Team
0,0,36,Stefan Remco Aartsen,M,25.0,194.0,78.0,NED,2000 Summer,2000,Summer,Sydney,Swimming,Swimming Men's 100 metres Butterfly,NaN,Netherlands
1,1,36,Stefan Remco Aartsen,M,25.0,194.0,78.0,NED,2000 Summer,2000,Summer,Sydney,Swimming,Swimming Men's 200 metres Butterfly,NaN,Netherlands
2,2,68,Pablo Martn Abal,M,23.0,178.0,76.0,ARG,2000 Summer,2000,Summer,Sydney,Swimming,Swimming Men's 100 metres Butterfly,NaN,Argentina
3,3,68,Pablo Martn Abal,M,23.0,178.0,76.0,ARG,2000 Summer,2000,Summer,Sydney,Swimming,Swimming Men's 4 x 100 metres Medley Relay,NaN,Argentina
4,4,120,Mohammed Abbas Sabih,M,26.0,170.0,70.0,IRQ,2004 Summer,2004,Summer,Athina,Swimming,Swimming Men's 100 metres Freestyle,NaN,Iraq


In [38]:
print(f"Rows with missing countries in male data: {male_merge['Team'].isna().sum()}")
print(f"Rows with missing countries in female data: {female_merge['Team'].isna().sum()}")

Rows with missing countries in male data: 0
Rows with missing countries in female data: 0


> Each NOC is now mapped to a single team

In [185]:
print(male_merge.loc[:, ['NOC', 'Team']].drop_duplicates()['NOC'].value_counts())

CZE    1
BOL    1
MEX    1
GUI    1
MDV    1
      ..
IVB    1
CPV    1
PRK    1
NGR    1
RUS    1
Name: NOC, Length: 210, dtype: int64


In [186]:
male_merge

,Unnamed: 0,ID,Name,Sex,Age,Height,Weight,NOC,Games,Year,Season,City,Sport,Event,Medal,Team
0,0,36,Stefan Remco Aartsen,M,25.0,194.0,78.0,NED,2000 Summer,2000,Summer,Sydney,Swimming,Swimming Men's 100 metres Butterfly,NaN,Netherlands
1,1,36,Stefan Remco Aartsen,M,25.0,194.0,78.0,NED,2000 Summer,2000,Summer,Sydney,Swimming,Swimming Men's 200 metres Butterfly,NaN,Netherlands
2,2,68,Pablo Martn Abal,M,23.0,178.0,76.0,ARG,2000 Summer,2000,Summer,Sydney,Swimming,Swimming Men's 100 metres Butterfly,NaN,Argentina
3,3,68,Pablo Martn Abal,M,23.0,178.0,76.0,ARG,2000 Summer,2000,Summer,Sydney,Swimming,Swimming Men's 4 x 100 metres Medley Relay,NaN,Argentina
4,4,120,Mohammed Abbas Sabih,M,26.0,170.0,70.0,IRQ,2004 Summer,2004,Summer,Athina,Swimming,Swimming Men's 100 metres Freestyle,NaN,Iraq
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16082,16851,134990,Misha Zilberman,M,27.0,171.0,61.0,ISR,2016 Summer,2016,Summer,Rio de Janeiro,Badminton,Badminton Men's Singles,NaN,Israel
16083,16852,135484,Nikolay Vladimirovich Zuyev,M,37.0,180.0,83.0,RUS,2004 Summer,2004,Summer,Athina,Badminton,Badminton Mixed Doubles,NaN,Russia
16084,16853,135528,Marc Zwiebler,M,24.0,181.0,75.0,GER,2008 Summer,2008,Summer,Beijing,Badminton,Badminton Men's Singles,NaN,Germany
16085,16854,135528,Marc Zwiebler,M,28.0,181.0,75.0,GER,2012 Summer,2012,Summer,London,Badminton,Badminton Men's Singles,NaN,Germany


# 4. Import and Clean GDP Data

In [40]:
def isNaN(num):    
    return num != num 
#Helper function to help merge GDP data

In [41]:
world_gdp = pd.read_csv('world_gdp_updated2.csv') #first 3 rows in the excel is empty
world_gdp

,Unnamed: 0,Country Name,Country Code,Year,GDP
0,0,Afghanistan,AFG,2000,NaN
1,1,Albania,ALB,2000,3632043908
2,2,Algeria,DZA,2000,54790245601
3,3,American Samoa,ASM,2000,4.99E+08
4,4,Andorra,AND,2000,1434429703
...,...,...,...,...,...
1102,1105,Taiwan,TWN,2000,3.31E+09
1103,1106,Taiwan,TWN,2004,3.47E+09
1104,1107,Taiwan,TWN,2008,4.16E+09
1105,1108,Taiwan,TWN,2012,4.96E+09


In [42]:
world_gdp.GDP.isna().sum()

44

In [43]:
print(len(list(set(male_merge['NOC'].unique()) - set(world_gdp['Country Code'].unique()))))
print("^Number of NOCs MALE that dont have a representation of GDP in the GDP dataset")

print(len(list(set(female_merge['NOC'].unique()) - set(world_gdp['Country Code'].unique()))))
print("^Number of NOCs FEMALE that dont have a representation of GDP in the GDP dataset")

88
^Number of NOCs MALE that dont have a representation of GDP in the GDP dataset
88
^Number of NOCs FEMALE that dont have a representation of GDP in the GDP dataset


In [44]:
print(len(list(set(male_merge['Team'].unique()) - set(world_gdp['Country Name'].unique()))))
print("^Number of MALE Teams that dont have a representation of GDP in the GDP dataset")

print(len(list(set(female_merge['Team'].unique()) - set(world_gdp['Country Name'].unique()))))
print("^Number of FEMALE Teams that dont have a representation of GDP in the GDP dataset")

3
^Number of MALE Teams that dont have a representation of GDP in the GDP dataset
3
^Number of FEMALE Teams that dont have a representation of GDP in the GDP dataset


In [45]:
for x in male_merge['Team'].unique():
    if x not in world_gdp['Country Name'].unique():
        print(x)

Refugee Olympic Athletes
Boliva
Individual Olympic Athletes


# 5. Merge with world_gdp DF using the country code added above

In [46]:
world_gdp.head()

,Unnamed: 0,Country Name,Country Code,Year,GDP
0,0,Afghanistan,AFG,2000,NaN
1,1,Albania,ALB,2000,3632043908
2,2,Algeria,DZA,2000,54790245601
3,3,American Samoa,ASM,2000,4.99E+08
4,4,Andorra,AND,2000,1434429703


 > Rename the Country Name to Team to merge

In [47]:
world_gdp.rename(columns = {'Country Name': 'Team'}, inplace = True)
world_gdp

,Unnamed: 0,Team,Country Code,Year,GDP
0,0,Afghanistan,AFG,2000,NaN
1,1,Albania,ALB,2000,3632043908
2,2,Algeria,DZA,2000,54790245601
3,3,American Samoa,ASM,2000,4.99E+08
4,4,Andorra,AND,2000,1434429703
...,...,...,...,...,...
1102,1105,Taiwan,TWN,2000,3.31E+09
1103,1106,Taiwan,TWN,2004,3.47E+09
1104,1107,Taiwan,TWN,2008,4.16E+09
1105,1108,Taiwan,TWN,2012,4.96E+09


In [48]:
male_gdp, female_gdp = pd.DataFrame(), pd.DataFrame()
male_merge_ccode = male_merge
female_merge_ccode = female_merge

In [49]:
for index, rows in male_merge_ccode.iterrows():
    if rows['Team'] == 'Boliva':
        male_merge_ccode.at[index, 'Team'] = 'Bolivia'

for index, rows in female_merge_ccode.iterrows():
    if rows['Team'] == 'Boliva':
        female_merge_ccode.at[index, 'Team'] = 'Bolivia'

In [50]:
male_gdp = male_merge_ccode.merge(world_gdp, left_on = ['Team', 'Year'], right_on = ['Team', 'Year'], how = 'left')
female_gdp = female_merge_ccode.merge(world_gdp, left_on = ['Team', 'Year'], right_on = ['Team', 'Year'], how = 'left')

> Drop Individual Olympic Athletes and Refugee Olympic Team, since they do not map to a single country

In [68]:
male_gdp.drop(male_gdp[male_gdp.Team == 'Individual Olympic Athletes'].index, inplace = True)
male_gdp.drop(male_gdp[male_gdp.Team == 'Refugee Olympic Athletes'].index, inplace = True)
female_gdp.drop(female_gdp[female_gdp.Team == 'Individual Olympic Athletes'].index, inplace = True)
female_gdp.drop(female_gdp[female_gdp.Team == 'Refugee Olympic Athletes'].index, inplace = True)

> Merge is complete, but there are still 63 rows with missing GDP values

In [69]:
world_gdp.loc[world_gdp['GDP'].isnull(), ['Team','Year']].drop_duplicates()

,Team,Year
0,Afghanistan,2000
75,Gibraltar,2000
145,Northern Mariana Islands,2000
147,Not classified,2000
172,Sint Maarten (Dutch part),2000
178,South Sudan,2000
183,St. Martin (French part),2000
201,Turks and Caicos Islands,2000
287,French Polynesia,2004
293,Gibraltar,2004


> Enumerate missing country and their respective years with missing GDP, see what is missing and fill them manually

In [70]:
dictFemale = {}
for index, rows in female_gdp.loc[female_gdp['GDP'].isnull(), ['Team','Year']].drop_duplicates().iterrows():
    if rows.Team not in dictFemale.keys():
        dictFemale[rows.Team] = []
    dictFemale[rows.Team].append(rows.Year)

In [63]:
dictFemale

{}

In [64]:
dictMale = {}
for index, rows in male_gdp.loc[male_gdp['GDP'].isnull(), ['Team','Year']].drop_duplicates().iterrows():
    if rows.Team not in dictMale.keys():
        dictMale[rows.Team] = []
    dictMale[rows.Team].append(rows.Year)

In [65]:
dictMale

{}

In [66]:
female_gdp.GDP.isna().sum()

0

In [67]:
male_gdp.loc[male_gdp['GDP'].isnull(), ['Team','Year','GDP']]

,Team,Year,GDP


> All rows have a GDP value now

In [206]:
# male_gdp.to_csv('male_gdp.csv')
# female_gdp.to_csv('female_gdp.csv')
# world_gdp.to_csv('world_gdp_clean.csv')

# 6. Import and Clean World Population data

In [82]:
world_pop = pd.read_csv('world_pop.csv')

In [83]:
world_pop.head()

,Country,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,101220.0,101353.0,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,9938414.0,...,26616792.0,27294031.0,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032
2,Angola,AGO,"Population, total",SP.POP.TOTL,5643182.0,5753024.0,5866061.0,5980417.0,6093321.0,6203299.0,...,20997687.0,21759420.0,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0,28813463
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,...,82683.0,83861.0,84462.0,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0,77281


> Clean world_pop.csv and melt year columns in to a single row

In [84]:
templist = [x for x in range(1960, 2000)]
templist = map(str, templist)
world_pop.drop(templist, axis = 1, inplace = True)

In [85]:
templist2 = [2001, 2002, 2003, 2005, 2006, 2007, 2009, 2010, 2011, 2013, 2014, 2015]
templist2 = map(str, templist2)
world_pop.drop(templist2, axis=1, inplace = True) 

In [86]:
world_pop

,Country,Country Code,Indicator Name,Indicator Code,2000,2004,2008,2012,2016
0,Aruba,ABW,"Population, total",SP.POP.TOTL,90853,98737.00,101353.0,102577.00,104822
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,20093756,24118979.00,27294031.0,30696958.00,34656032
2,Angola,AGO,"Population, total",SP.POP.TOTL,16440924,18865716.00,21759420.0,25096150.00,28813463
3,Albania,ALB,"Population, total",SP.POP.TOTL,3089027,3026939.00,2947314.0,2900401.00,2876101
4,Andorra,AND,"Population, total",SP.POP.TOTL,65390,76244.00,83861.0,82431.00,77281
...,...,...,...,...,...,...,...,...,...
212,Kosovo,XKX,"Population, total",SP.POP.TOTL,1700000,1704622.00,1747383.0,1805200.00,1816200
213,"Yemen, Rep.",YEM,"Population, total",SP.POP.TOTL,17874725,20017068.00,22356391.0,24909969.00,27584213
214,South Africa,ZAF,"Population, total",SP.POP.TOTL,44896856,47001700.99,49557573.3,52506515.08,55908865
215,Zambia,ZMB,"Population, total",SP.POP.TOTL,10531221,11731746.00,13082517.0,14699937.00,16591390


In [87]:
world_pop = pd.melt(world_pop, id_vars = ['Country', 'Country Code','Indicator Name', 'Indicator Code'], var_name = 'Year', value_name = 'Population')

In [88]:
world_pop['Year'] = pd.to_numeric(world_pop['Year'])

In [89]:
world_pop

,Country,Country Code,Indicator Name,Indicator Code,Year,Population
0,Aruba,ABW,"Population, total",SP.POP.TOTL,2000,90853.0
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,2000,20093756.0
2,Angola,AGO,"Population, total",SP.POP.TOTL,2000,16440924.0
3,Albania,ALB,"Population, total",SP.POP.TOTL,2000,3089027.0
4,Andorra,AND,"Population, total",SP.POP.TOTL,2000,65390.0
...,...,...,...,...,...,...
1080,Kosovo,XKX,"Population, total",SP.POP.TOTL,2016,1816200.0
1081,"Yemen, Rep.",YEM,"Population, total",SP.POP.TOTL,2016,27584213.0
1082,South Africa,ZAF,"Population, total",SP.POP.TOTL,2016,55908865.0
1083,Zambia,ZMB,"Population, total",SP.POP.TOTL,2016,16591390.0


In [90]:
world_pop.Population.isna().sum()

0

In [91]:
print(len(list(set(male_gdp['Team'].unique()) - set(world_pop['Country'].unique()))))
print("^Number of MALE Teams that dont have a representation of GDP in the GDP dataset")

print(len(list(set(female_gdp['Team'].unique()) - set(world_pop['Country'].unique()))))
print("^Number of FEMALE Teams that dont have a representation of GDP in the GDP dataset")

30
^Number of MALE Teams that dont have a representation of GDP in the GDP dataset
30
^Number of FEMALE Teams that dont have a representation of GDP in the GDP dataset


In [92]:
world_pop.rename(columns = {'Country': 'Team'}, inplace = True)

# 7.Merge world_pop with male_complete and female_complete, and manually fill out those countries with missing POPULATION

In [93]:

male_complete = male_gdp.merge(world_pop, left_on = ['Team', 'Year'], right_on = ['Team', 'Year'], how = 'left')
female_complete = female_gdp.merge(world_pop, left_on = ['Team', 'Year'], right_on = ['Team', 'Year'], how = 'left')

In [94]:
dictMalePop = {}
for index, rows in male_complete.loc[male_complete['Population'].isnull(), ['Team','Year']].drop_duplicates().iterrows():
    if rows.Team not in dictMalePop.keys():
        dictMalePop[rows.Team] = []
    dictMalePop[rows.Team].append(rows.Year)
dictMalePop

{'Palestine': [2008, 2004, 2012, 2016, 2000],
 'Venezuela': [2008, 2012, 2016, 2000, 2004],
 'Saint Vincent': [2012, 2004, 2016, 2000, 2008],
 'United Arab Emirates': [2012, 2004, 2008, 2000, 2016],
 'Yemen': [2008, 2004, 2012, 2000, 2016],
 'Kyrgyzstan': [2012, 2000, 2004, 2008, 2016],
 'Ivory Coast': [2000, 2004, 2008, 2012, 2016],
 'Saint Lucia': [2016, 2000, 2012, 2008, 2004],
 'Micronesia': [2016, 2004, 2008, 2012, 2000],
 'Republic of Congo': [2004, 2008, 2012, 2016, 2000],
 'Syria': [2012, 2016, 2004, 2008, 2000],
 'Macedonia': [2012, 2016, 2000, 2004, 2008],
 'Trinidad': [2000, 2004, 2008, 2012, 2016],
 'Bahamas': [2008, 2000, 2004, 2016, 2012],
 'Taiwan': [2004, 2008, 2012, 2000, 2016],
 'Laos': [2008, 2012, 2016, 2000, 2004],
 'South Korea': [2012, 2016, 2000, 2004, 2008],
 'Virgin Islands, US': [2000, 2004, 2008, 2016, 2012],
 'Gambia': [2016, 2000, 2004, 2008, 2012],
 'Democratic Republic of the Congo': [2008, 2000, 2004, 2016, 2012],
 'Slovakia': [2012, 2016, 2004, 2008, 2

> Manually filling out missing population data from the above countries

In [220]:
for index, rows in male_complete.iterrows():
    
    if rows['Team'] == 'Palestine' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  3224003
    elif rows['Team'] == 'Palestine' and rows.Year == 2004:
        male_complete.at[index, 'Population'] =  3505323
    elif rows['Team'] == 'Palestine' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  3856679
    elif rows['Team'] == 'Palestine' and rows.Year == 2012:
        male_complete.at[index, 'Population'] =  4241473
    elif rows['Team'] == 'Palestine' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  24635654
        
        
    elif rows['Team'] == 'Venezuela' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  24200000
    elif rows['Team'] == 'Venezuela' and rows.Year == 2004:
        male_complete.at[index, 'Population'] =  26000000
    elif rows['Team'] == 'Venezuela' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  27600000
    elif rows['Team'] == 'Venezuela' and rows.Year == 2012:
        male_complete.at[index, 'Population'] =  29400000
    elif rows['Team'] == 'Venezuela' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  29900000

        
    elif rows['Team'] == 'Saint Vincent' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  107787
    elif rows['Team'] == 'Saint Vincent' and rows.Year == 2004:
        male_complete.at[index, 'Population'] =  108520
    elif rows['Team'] == 'Saint Vincent' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  108401
    elif rows['Team'] == 'Saint Vincent' and rows.Year == 2012:
        male_complete.at[index, 'Population'] =  108435
    elif rows['Team'] == 'Saint Vincent' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  109467
        
    elif rows['Team'] == 'United Arab Emirates' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  3134000
    elif rows['Team'] == 'United Arab Emirates' and rows.Year == 2004:
        male_complete.at[index, 'Population'] =  4069000
    elif rows['Team'] == 'United Arab Emirates' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  70890000
    elif rows['Team'] == 'United Arab Emirates' and rows.Year == 2012:
        male_complete.at[index, 'Population'] =  9142000
    elif rows['Team'] == 'United Arab Emirates' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  9361000

    elif rows['Team'] == 'Yemen' and rows.Year == 2004:
        male_complete.at[index, 'Population'] =  19500000
    elif rows['Team'] == 'Yemen' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  21900000
    elif rows['Team'] == 'Yemen' and rows.Year == 2012:
        male_complete.at[index, 'Population'] =  24500000
    elif rows['Team'] == 'Yemen' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  29400000
    elif rows['Team'] == 'Yemen' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  29900000
             
    elif rows['Team'] == 'Ivory Coast' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  16450000
    elif rows['Team'] == 'Ivory Coast' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  17970000
    elif rows['Team'] == 'Ivory Coast' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  19610000
    elif rows['Team'] == 'Ivory Coast' and rows.Year == 2012:
        male_complete.at[index, 'Population'] =  21550000
    elif rows['Team'] == 'Ivory Coast' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  23820000
        
        
    elif rows['Team'] == 'Kyrgyzstan' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  4898000
    elif rows['Team'] == 'Kyrgyzstan' and rows.Year == 2004:
        male_complete.at[index, 'Population'] =  5105000
    elif rows['Team'] == 'Kyrgyzstan' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  5319000
    elif rows['Team'] == 'Kyrgyzstan' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 5607000
    elif rows['Team'] == 'Kyrgyzstan' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  5957000
        
    elif rows['Team'] == 'Saint Lucia' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  156737
    elif rows['Team'] == 'Saint Lucia' and rows.Year == 2004:
        male_complete.at[index, 'Population'] =  161821
    elif rows['Team'] == 'Saint Lucia' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  170011
    elif rows['Team'] == 'Saint Lucia' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 176654
    elif rows['Team'] == 'Saint Lucia' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  180028
        
    elif rows['Team'] == 'Micronesia' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  484775
    elif rows['Team'] == 'Micronesia' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  496716
    elif rows['Team'] == 'Micronesia' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  500734
    elif rows['Team'] == 'Micronesia' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 509680
    elif rows['Team'] == 'Micronesia' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  527813
        
    elif rows['Team'] == 'Republic of Congo' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  3127000
    elif rows['Team'] == 'Republic of Congo' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  3510000
    elif rows['Team'] == 'Republic of Congo' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  4011000
    elif rows['Team'] == 'Republic of Congo' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 4510000
    elif rows['Team'] == 'Republic of Congo' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  4981000
        
    elif rows['Team'] == 'Syria' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  16410000
    elif rows['Team'] == 'Syria' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  17830000
    elif rows['Team'] == 'Syria' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  500734
    elif rows['Team'] == 'Syria' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 28390000
    elif rows['Team'] == 'Syria' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  36610000
        
    elif rows['Team'] == 'Macedonia' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  2035000
    elif rows['Team'] == 'Macedonia' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  2057000
    elif rows['Team'] == 'Macedonia' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  2067000
    elif rows['Team'] == 'Macedonia' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 2074000
    elif rows['Team'] == 'Macedonia' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  2081000
        
    elif rows['Team'] == 'Trinidad' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  1267000
    elif rows['Team'] == 'Trinidad' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  1290000
    elif rows['Team'] == 'Trinidad' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  1314000
    elif rows['Team'] == 'Trinidad' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 1345000
    elif rows['Team'] == 'Trinidad' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  1378000
        
    elif rows['Team'] == 'Bahamas' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  298045
    elif rows['Team'] == 'Bahamas' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  318893
    elif rows['Team'] == 'Bahamas' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  343680
    elif rows['Team'] == 'Bahamas' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 363581
    elif rows['Team'] == 'Bahamas' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  377923
        
    elif rows['Team'] == 'Taiwan' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  21966527
    elif rows['Team'] == 'Taiwan' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  22570224
    elif rows['Team'] == 'Taiwan' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  23019045
    elif rows['Team'] == 'Taiwan' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 23346896
    elif rows['Team'] == 'Taiwan' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  23610200
        
    elif rows['Team'] == 'Laos' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  5324000
    elif rows['Team'] == 'Laos' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  5624000
    elif rows['Team'] == 'Laos' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  6047000
    elif rows['Team'] == 'Laos' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 6445000
    elif rows['Team'] == 'Laos' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  6846000
        
    elif rows['Team'] == 'South Korea' and rows.Year == 2000:
        male_complete.at[index, 'Population'] = 47010000
    elif rows['Team'] == 'South Korea' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  48080000
    elif rows['Team'] == 'South Korea' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  49050000
    elif rows['Team'] == 'South Korea' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 50200000
    elif rows['Team'] == 'South Korea' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  51200000
    
    elif rows['Team'] == 'Virgin Islands, US' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  108642
    elif rows['Team'] == 'Virgin Islands, US' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  108466
    elif rows['Team'] == 'Virgin Islands, US' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  108397
    elif rows['Team'] == 'Virgin Islands, US' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 108188
    elif rows['Team'] == 'Virgin Islands, US' and rows.Year == 2016:
            male_complete.at[index, 'Population'] =  107516
        
    elif rows['Team'] == 'Gambia' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =1318000
    elif rows['Team'] == 'Gambia' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  1497000
    elif rows['Team'] == 'Gambia' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  1689000
    elif rows['Team'] == 'Gambia' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 1905000
    elif rows['Team'] == 'Gambia' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  2149000
        
    elif rows['Team'] == 'Democratic Republic of the Congo' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  48430000
    elif rows['Team'] == 'Democratic Republic of the Congo' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  53070000
    elif rows['Team'] == 'Democratic Republic of the Congo' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  60410000
    elif rows['Team'] == 'Democratic Republic of the Congo' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 69020000
    elif rows['Team'] == 'Democratic Republic of the Congo' and rows.Year == 2016:
        male_complete.at[index, 'Population'] = 78790000
        
    elif rows['Team'] == 'Slovakia' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  5389000
    elif rows['Team'] == 'Slovakia' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  5372000
    elif rows['Team'] == 'Slovakia' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  5379000
    elif rows['Team'] == 'Slovakia' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 5398000
    elif rows['Team'] == 'Slovakia' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  5431000
        
    elif rows['Team'] == 'Brunei' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  333186
    elif rows['Team'] == 'Brunei' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  359434
    elif rows['Team'] == 'Brunei' and rows.Year == 2008:
         male_complete.at[index, 'Population'] =  379148
    elif rows['Team'] == 'Brunei' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 398997
    elif rows['Team'] == 'Brunei' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  419791
        
    elif rows['Team'] == 'Antigua' and rows.Year == 2000:
        male_complete.at[index, 'Population'] = 76007 
    elif rows['Team'] == 'Antigua' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  80347
    elif rows['Team'] == 'Antigua' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  85394
    elif rows['Team'] == 'Antigua' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 90407
    elif rows['Team'] == 'Antigua' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  94520
        
    elif rows['Team'] == 'Cook Islands' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  17930
    elif rows['Team'] == 'Cook Islands' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  18904
    elif rows['Team'] == 'Cook Islands' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  18818
    elif rows['Team'] == 'Cook Islands' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 18011
    elif rows['Team'] == 'Cook Islands' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  17524
        
    elif rows['Team'] == 'Eritrea' and rows.Year == 2000:
        male_complete.at[index, 'Population'] = 2292416 
    elif rows['Team'] == 'Eritrea' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  2719803
    elif rows['Team'] == 'Eritrea' and rows.Year == 2008:
         male_complete.at[index, 'Population'] =  3062779
    elif rows['Team'] == 'Eritrea' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 3250100
    elif rows['Team'] == 'Eritrea' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  3376557
        
    elif rows['Team'] == 'Saint Kitts' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  44083
    elif rows['Team'] == 'Saint Kitts' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  46323
    elif rows['Team'] == 'Saint Kitts' and rows.Year == 2008:
         male_complete.at[index, 'Population'] =  48178
    elif rows['Team'] == 'Saint Kitts' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 49881
    elif rows['Team'] == 'Saint Kitts' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  51629
        
    elif rows['Team'] == 'Virgin Islands, British' and rows.Year == 2000:
        male_complete.at[index, 'Population'] = 20131
    elif rows['Team'] == 'Virgin Islands, British' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  22329
    elif rows['Team'] == 'Virgin Islands, British' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  26096
    elif rows['Team'] == 'Virgin Islands, British' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 28654
    elif rows['Team'] == 'Virgin Islands, British' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  29355
        
    elif rows['Team'] == 'Cape Verde' and rows.Year == 2000:
        male_complete.at[index, 'Population'] =  428178
    elif rows['Team'] == 'Cape Verde' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  456619
    elif rows['Team'] == 'Cape Verde' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  480846
    elif rows['Team'] == 'Cape Verde' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 505241
    elif rows['Team'] == 'Cape Verde' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  531140
        
    elif rows['Team'] == 'North Korea' and rows.Year == 2000:
        male_complete.at[index, 'Population'] = 22390000
    elif rows['Team'] == 'North Korea' and rows.Year == 2004:
         male_complete.at[index, 'Population'] =  23730000
    elif rows['Team'] == 'North Korea' and rows.Year == 2008:
        male_complete.at[index, 'Population'] =  24310000
    elif rows['Team'] == 'North Korea' and rows.Year == 2012:
        male_complete.at[index, 'Population'] = 24800000
    elif rows['Team'] == 'North Korea' and rows.Year == 2016:
        male_complete.at[index, 'Population'] =  25310000

    elif rows['Team'] == 'South Sudan' and rows.Year == 2016:
        male_complete.at[index, 'Population'] = 10830000

In [221]:
dictMalePop2 = {}
for index, rows in male_complete.loc[male_complete['Population'].isnull(), ['Team','Year']].drop_duplicates().iterrows():
    if rows.Team not in dictMalePop2.keys():
        dictMalePop2[rows.Team] = []
    dictMalePop2[rows.Team].append(rows.Year)
dictMalePop2

{}

In [222]:
for index, rows in female_complete.iterrows():
    
    if rows['Team'] == 'Palestine' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  3224003
    elif rows['Team'] == 'Palestine' and rows.Year == 2004:
        female_complete.at[index, 'Population'] =  3505323
    elif rows['Team'] == 'Palestine' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  3856679
    elif rows['Team'] == 'Palestine' and rows.Year == 2012:
        female_complete.at[index, 'Population'] =  4241473
    elif rows['Team'] == 'Palestine' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  24635654
        
        
    elif rows['Team'] == 'Venezuela' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  24200000
    elif rows['Team'] == 'Venezuela' and rows.Year == 2004:
        female_complete.at[index, 'Population'] =  26000000
    elif rows['Team'] == 'Venezuela' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  27600000
    elif rows['Team'] == 'Venezuela' and rows.Year == 2012:
        female_complete.at[index, 'Population'] =  29400000
    elif rows['Team'] == 'Venezuela' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  29900000

        
    elif rows['Team'] == 'Saint Vincent' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  107787
    elif rows['Team'] == 'Saint Vincent' and rows.Year == 2004:
        female_complete.at[index, 'Population'] =  108520
    elif rows['Team'] == 'Saint Vincent' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  108401
    elif rows['Team'] == 'Saint Vincent' and rows.Year == 2012:
        female_complete.at[index, 'Population'] =  108435
    elif rows['Team'] == 'Saint Vincent' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  109467
        
    elif rows['Team'] == 'United Arab Emirates' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  3134000
    elif rows['Team'] == 'United Arab Emirates' and rows.Year == 2004:
        female_complete.at[index, 'Population'] =  4069000
    elif rows['Team'] == 'United Arab Emirates' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  70890000
    elif rows['Team'] == 'United Arab Emirates' and rows.Year == 2012:
        female_complete.at[index, 'Population'] =  9142000
    elif rows['Team'] == 'United Arab Emirates' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  9361000

    elif rows['Team'] == 'Yemen' and rows.Year == 2004:
        female_complete.at[index, 'Population'] =  19500000
    elif rows['Team'] == 'Yemen' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  21900000
    elif rows['Team'] == 'Yemen' and rows.Year == 2012:
        female_complete.at[index, 'Population'] =  24500000
    elif rows['Team'] == 'Yemen' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  29400000
    elif rows['Team'] == 'Yemen' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  29900000
             
    elif rows['Team'] == 'Ivory Coast' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  16450000
    elif rows['Team'] == 'Ivory Coast' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  17970000
    elif rows['Team'] == 'Ivory Coast' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  19610000
    elif rows['Team'] == 'Ivory Coast' and rows.Year == 2012:
        female_complete.at[index, 'Population'] =  21550000
    elif rows['Team'] == 'Ivory Coast' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  23820000
        
        
    elif rows['Team'] == 'Kyrgyzstan' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  4898000
    elif rows['Team'] == 'Kyrgyzstan' and rows.Year == 2004:
        female_complete.at[index, 'Population'] =  5105000
    elif rows['Team'] == 'Kyrgyzstan' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  5319000
    elif rows['Team'] == 'Kyrgyzstan' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 5607000
    elif rows['Team'] == 'Kyrgyzstan' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  5957000
        
    elif rows['Team'] == 'Saint Lucia' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  156737
    elif rows['Team'] == 'Saint Lucia' and rows.Year == 2004:
        female_complete.at[index, 'Population'] =  161821
    elif rows['Team'] == 'Saint Lucia' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  170011
    elif rows['Team'] == 'Saint Lucia' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 176654
    elif rows['Team'] == 'Saint Lucia' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  180028
        
    elif rows['Team'] == 'Micronesia' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  484775
    elif rows['Team'] == 'Micronesia' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  496716
    elif rows['Team'] == 'Micronesia' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  500734
    elif rows['Team'] == 'Micronesia' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 509680
    elif rows['Team'] == 'Micronesia' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  527813
        
    elif rows['Team'] == 'Republic of Congo' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  3127000
    elif rows['Team'] == 'Republic of Congo' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  3510000
    elif rows['Team'] == 'Republic of Congo' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  4011000
    elif rows['Team'] == 'Republic of Congo' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 4510000
    elif rows['Team'] == 'Republic of Congo' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  4981000
        
    elif rows['Team'] == 'Syria' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  16410000
    elif rows['Team'] == 'Syria' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  17830000
    elif rows['Team'] == 'Syria' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  500734
    elif rows['Team'] == 'Syria' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 28390000
    elif rows['Team'] == 'Syria' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  36610000
        
    elif rows['Team'] == 'Macedonia' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  2035000
    elif rows['Team'] == 'Macedonia' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  2057000
    elif rows['Team'] == 'Macedonia' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  2067000
    elif rows['Team'] == 'Macedonia' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 2074000
    elif rows['Team'] == 'Macedonia' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  2081000
        
    elif rows['Team'] == 'Trinidad' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  1267000
    elif rows['Team'] == 'Trinidad' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  1290000
    elif rows['Team'] == 'Trinidad' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  1314000
    elif rows['Team'] == 'Trinidad' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 1345000
    elif rows['Team'] == 'Trinidad' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  1378000
        
    elif rows['Team'] == 'Bahamas' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  298045
    elif rows['Team'] == 'Bahamas' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  318893
    elif rows['Team'] == 'Bahamas' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  343680
    elif rows['Team'] == 'Bahamas' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 363581
    elif rows['Team'] == 'Bahamas' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  377923
        
    elif rows['Team'] == 'Taiwan' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  21966527
    elif rows['Team'] == 'Taiwan' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  22570224
    elif rows['Team'] == 'Taiwan' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  23019045
    elif rows['Team'] == 'Taiwan' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 23346896
    elif rows['Team'] == 'Taiwan' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  23610200
        
    elif rows['Team'] == 'Laos' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  5324000
    elif rows['Team'] == 'Laos' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  5624000
    elif rows['Team'] == 'Laos' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  6047000
    elif rows['Team'] == 'Laos' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 6445000
    elif rows['Team'] == 'Laos' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  6846000
        
    elif rows['Team'] == 'South Korea' and rows.Year == 2000:
        female_complete.at[index, 'Population'] = 47010000
    elif rows['Team'] == 'South Korea' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  48080000
    elif rows['Team'] == 'South Korea' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  49050000
    elif rows['Team'] == 'South Korea' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 50200000
    elif rows['Team'] == 'South Korea' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  51200000
    
    elif rows['Team'] == 'Virgin Islands, US' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  108642
    elif rows['Team'] == 'Virgin Islands, US' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  108466
    elif rows['Team'] == 'Virgin Islands, US' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  108397
    elif rows['Team'] == 'Virgin Islands, US' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 108188
    elif rows['Team'] == 'Virgin Islands, US' and rows.Year == 2016:
            female_complete.at[index, 'Population'] =  107516
        
    elif rows['Team'] == 'Gambia' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =1318000
    elif rows['Team'] == 'Gambia' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  1497000
    elif rows['Team'] == 'Gambia' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  1689000
    elif rows['Team'] == 'Gambia' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 1905000
    elif rows['Team'] == 'Gambia' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  2149000
        
    elif rows['Team'] == 'Democratic Republic of the Congo' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  48430000
    elif rows['Team'] == 'Democratic Republic of the Congo' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  53070000
    elif rows['Team'] == 'Democratic Republic of the Congo' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  60410000
    elif rows['Team'] == 'Democratic Republic of the Congo' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 69020000
    elif rows['Team'] == 'Democratic Republic of the Congo' and rows.Year == 2016:
        female_complete.at[index, 'Population'] = 78790000
        
    elif rows['Team'] == 'Slovakia' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  5389000
    elif rows['Team'] == 'Slovakia' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  5372000
    elif rows['Team'] == 'Slovakia' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  5379000
    elif rows['Team'] == 'Slovakia' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 5398000
    elif rows['Team'] == 'Slovakia' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  5431000
        
    elif rows['Team'] == 'Brunei' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  333186
    elif rows['Team'] == 'Brunei' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  359434
    elif rows['Team'] == 'Brunei' and rows.Year == 2008:
         female_complete.at[index, 'Population'] =  379148
    elif rows['Team'] == 'Brunei' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 398997
    elif rows['Team'] == 'Brunei' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  419791
        
    elif rows['Team'] == 'Antigua' and rows.Year == 2000:
        female_complete.at[index, 'Population'] = 76007 
    elif rows['Team'] == 'Antigua' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  80347
    elif rows['Team'] == 'Antigua' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  85394
    elif rows['Team'] == 'Antigua' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 90407
    elif rows['Team'] == 'Antigua' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  94520
        
    elif rows['Team'] == 'Cook Islands' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  17930
    elif rows['Team'] == 'Cook Islands' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  18904
    elif rows['Team'] == 'Cook Islands' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  18818
    elif rows['Team'] == 'Cook Islands' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 18011
    elif rows['Team'] == 'Cook Islands' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  17524
        
    elif rows['Team'] == 'Eritrea' and rows.Year == 2000:
        female_complete.at[index, 'Population'] = 2292416 
    elif rows['Team'] == 'Eritrea' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  2719803
    elif rows['Team'] == 'Eritrea' and rows.Year == 2008:
         female_complete.at[index, 'Population'] =  3062779
    elif rows['Team'] == 'Eritrea' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 3250100
    elif rows['Team'] == 'Eritrea' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  3376557
        
    elif rows['Team'] == 'Saint Kitts' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  44083
    elif rows['Team'] == 'Saint Kitts' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  46323
    elif rows['Team'] == 'Saint Kitts' and rows.Year == 2008:
         female_complete.at[index, 'Population'] =  48178
    elif rows['Team'] == 'Saint Kitts' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 49881
    elif rows['Team'] == 'Saint Kitts' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  51629
        
    elif rows['Team'] == 'Virgin Islands, British' and rows.Year == 2000:
        female_complete.at[index, 'Population'] = 20131
    elif rows['Team'] == 'Virgin Islands, British' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  22329
    elif rows['Team'] == 'Virgin Islands, British' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  26096
    elif rows['Team'] == 'Virgin Islands, British' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 28654
    elif rows['Team'] == 'Virgin Islands, British' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  29355
        
    elif rows['Team'] == 'Cape Verde' and rows.Year == 2000:
        female_complete.at[index, 'Population'] =  428178
    elif rows['Team'] == 'Cape Verde' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  456619
    elif rows['Team'] == 'Cape Verde' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  480846
    elif rows['Team'] == 'Cape Verde' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 505241
    elif rows['Team'] == 'Cape Verde' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  531140
        
    elif rows['Team'] == 'North Korea' and rows.Year == 2000:
        female_complete.at[index, 'Population'] = 22390000
    elif rows['Team'] == 'North Korea' and rows.Year == 2004:
         female_complete.at[index, 'Population'] =  23730000
    elif rows['Team'] == 'North Korea' and rows.Year == 2008:
        female_complete.at[index, 'Population'] =  24310000
    elif rows['Team'] == 'North Korea' and rows.Year == 2012:
        female_complete.at[index, 'Population'] = 24800000
    elif rows['Team'] == 'North Korea' and rows.Year == 2016:
        female_complete.at[index, 'Population'] =  25310000

    elif rows['Team'] == 'South Sudan' and rows.Year == 2016:
        female_complete.at[index, 'Population'] = 10830000

In [223]:
dictFemalePop = {}
for index, rows in female_complete.loc[male_complete['Population'].isnull(), ['Team','Year']].drop_duplicates().iterrows():
    if rows.Team not in dictFemalePop.keys():
        dictFemalePop[rows.Team] = []
    dictFemalePop[rows.Team].append(rows.Year)
dictFemalePop

{}

> Creating new column GDP per capita

In [224]:
male_complete.at[9667, 'GDP'] = 27631e+07
male_complete.at[14151, 'GDP'] = 27631e+07
# Manually correcting wrong wrongly typed values

In [225]:
male_complete['GDP'] = pd.to_numeric(male_complete['GDP'])
male_complete['Population'] = pd.to_numeric(male_complete['Population'])
male_completed = male_complete

male_completed['GDP Per Capita'] = male_complete['GDP'] / male_complete['Population']

In [226]:
female_complete.at[11094, 'GDP'] = 27631e+07
# Manually correcting wrong wrongly typed values

In [227]:
female_complete['GDP'] = pd.to_numeric(female_complete['GDP'])
female_complete['Population'] = pd.to_numeric(female_complete['Population'])
female_completed = female_complete

female_completed['GDP Per Capita'] = female_complete['GDP'] / female_complete['Population'] 

> All should be completed now, with no missing values

In [228]:
display(male_completed.head(), female_completed.head())

,Unnamed: 0_x,ID,Name,Sex,Age,Height,Weight,NOC,Games,Year,...,Medal,Team,Unnamed: 0_y,Country Code_x,GDP,Country Code_y,Indicator Name,Indicator Code,Population,GDP Per Capita
0,0,36,Stefan Remco Aartsen,M,25.0,194.0,78.0,NED,2000 Summer,2000,...,NaN,Netherlands,140.0,NLD,4.130000e+11,NLD,"Population, total",SP.POP.TOTL,15925513.0,25933.230534
1,1,36,Stefan Remco Aartsen,M,25.0,194.0,78.0,NED,2000 Summer,2000,...,NaN,Netherlands,140.0,NLD,4.130000e+11,NLD,"Population, total",SP.POP.TOTL,15925513.0,25933.230534
2,2,68,Pablo Martn Abal,M,23.0,178.0,76.0,ARG,2000 Summer,2000,...,NaN,Argentina,7.0,ARG,2.840000e+11,ARG,"Population, total",SP.POP.TOTL,37057452.0,7663.775696
3,3,68,Pablo Martn Abal,M,23.0,178.0,76.0,ARG,2000 Summer,2000,...,NaN,Argentina,7.0,ARG,2.840000e+11,ARG,"Population, total",SP.POP.TOTL,37057452.0,7663.775696
4,4,120,Mohammed Abbas Sabih,M,26.0,170.0,70.0,IRQ,2004 Summer,2004,...,NaN,Iraq,311.0,IRQ,3.662790e+10,IRQ,"Population, total",SP.POP.TOTL,26316609.0,1391.816923


,Unnamed: 0_x,ID,Name,Sex,Age,Height,Weight,NOC,Games,Year,...,Medal,Team,Unnamed: 0_y,Country Code_x,GDP,Country Code_y,Indicator Name,Indicator Code,Population,GDP Per Capita
0,0,290,Fatima Abdul Majeed Hameed Al-Kirashi,F,14.0,173.953608,64.128866,BRN,2000 Summer,2000,...,NaN,Bahrain,14.0,BHR,9.062907e+09,BHR,"Population, total",SP.POP.TOTL,664614.0,13636.346684
1,1,799,Nafissatou Moussa Adamou,F,14.0,172.000000,72.000000,NIG,2012 Summer,2012,...,NaN,Niger,801.0,NER,6.942210e+09,NER,"Population, total",SP.POP.TOTL,17731634.0,391.515502
2,2,811,Natalie Cammile Adams (-Brannan),F,20.0,173.000000,65.000000,USA,2012 Summer,2012,...,NaN,USA,865.0,USA,1.620000e+13,USA,"Population, total",SP.POP.TOTL,313998379.0,51592.623031
3,3,811,Natalie Cammile Adams (-Brannan),F,24.0,173.000000,65.000000,USA,2016 Summer,2016,...,NaN,USA,1084.0,USA,1.860000e+13,USA,"Population, total",SP.POP.TOTL,323127513.0,57562.414996
4,4,868,"Frances ""Fran"" Adcock",F,20.0,178.000000,72.000000,AUS,2004 Summer,2004,...,NaN,Australia,229.0,AUS,6.130000e+11,AUS,"Population, total",SP.POP.TOTL,20127400.0,30455.995310


In [229]:
print(f"Number of NULL GDP Per Capita: {male_completed['GDP Per Capita'].isna().sum()} ")
print(f"Number of NULL GDP Per Capita: {female_completed['GDP Per Capita'].isna().sum()}")

Number of NULL GDP Per Capita: 0 
Number of NULL GDP Per Capita: 0
